## ------------------------ D A T A --- E X P L O R A T I O N ------------------------

### We start by importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import requests
import pycountry
import re
import json

%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from datetime import datetime
from random import randint

init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
# We tweek the amount of rows that pandas dataframes can display
pd.options.display.max_rows = 999

In [3]:
# We import the original dataset into a pandas dataframe
df = pd.read_csv('datasets/AccidentesAviones.csv')
df.head()

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


In [4]:
df.shape, df.columns

((5008, 18),
 Index(['Unnamed: 0', 'fecha', 'HORA declarada', 'Ruta', 'OperadOR',
        'flight_no', 'route', 'ac_type', 'registration', 'cn_ln', 'all_aboard',
        'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos',
        'passenger_fatalities', 'crew_fatalities', 'ground', 'summary'],
       dtype='object'))

In [5]:
# We check whether the 'Unnamed: 0' column has any valuable information (spoiler: it doesn't)
df.drop('Unnamed: 0', axis=1).duplicated().value_counts()

False    5008
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              5008 non-null   int64 
 1   fecha                   5008 non-null   object
 2   HORA declarada          5008 non-null   object
 3   Ruta                    5008 non-null   object
 4   OperadOR                5008 non-null   object
 5   flight_no               5008 non-null   object
 6   route                   5008 non-null   object
 7   ac_type                 5008 non-null   object
 8   registration            5008 non-null   object
 9   cn_ln                   5008 non-null   object
 10  all_aboard              5008 non-null   object
 11  PASAJEROS A BORDO       5008 non-null   object
 12  crew_aboard             5008 non-null   object
 13  cantidad de fallecidos  5008 non-null   object
 14  passenger_fatalities    5008 non-null   object
 15  crew

Now we will delete the 'Unnamed: 0' column, as it contains a unique identifier for each row and is not needed yet (we will generate a new index after dropping the row that we need to drop). We will also rename the others columns in order to make more clear what information is contained on each of them.

Both of this changes will be performed in a copy of the original dataset so we can preserve the original data and access it every time we need to.

In [7]:
df2 = df.copy()

In [8]:
df2.columns

Index(['Unnamed: 0', 'fecha', 'HORA declarada', 'Ruta', 'OperadOR',
       'flight_no', 'route', 'ac_type', 'registration', 'cn_ln', 'all_aboard',
       'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos',
       'passenger_fatalities', 'crew_fatalities', 'ground', 'summary'],
      dtype='object')

In [9]:
new_col_names = {'fecha':'date',
                 'HORA declarada': 'time',
                 'Ruta': 'accident_location',
                 'OperadOR': 'operator',
                 'all_aboard': 'total_aboard',
                 'PASAJEROS A BORDO': 'passengers_aboard',
                 'crew_aboard': 'crew_aboard',
                 'cantidad de fallecidos': 'total_fatalities',
                 'ground': 'ground_fatalities'}

df2.rename(columns = new_col_names, inplace = True)
df2.columns

Index(['Unnamed: 0', 'date', 'time', 'accident_location', 'operator',
       'flight_no', 'route', 'ac_type', 'registration', 'cn_ln',
       'total_aboard', 'passengers_aboard', 'crew_aboard', 'total_fatalities',
       'passenger_fatalities', 'crew_fatalities', 'ground_fatalities',
       'summary'],
      dtype='object')

In [10]:
df2.drop('Unnamed: 0', axis=1, inplace=True)
df2.columns

Index(['date', 'time', 'accident_location', 'operator', 'flight_no', 'route',
       'ac_type', 'registration', 'cn_ln', 'total_aboard', 'passengers_aboard',
       'crew_aboard', 'total_fatalities', 'passenger_fatalities',
       'crew_fatalities', 'ground_fatalities', 'summary'],
      dtype='object')

## Per column data transformations

In [11]:
# Here we can get an idea of the distribution of values in each column
# The output is too large to execute in the notebook, but here we can notice some strange values such as '?' appearing in several columns
'''
for col in df2:
    print(f'----------{col}----------\n{df[col].value_counts()}\n\n')
'''

"\nfor col in df2:\n    print(f'----------{col}----------\n{df[col].value_counts()}\n\n')\n"

In [12]:
# As we noticed that some columns have '?' imputed wherever a value is missing, we count the real missing values
for x in df2:
    res = False
    cnt = 0
    for y in df2[x]:
        if y == '?':
            res = True
            cnt += 1
    print(f"{x} contains '?': {res} ({cnt})")

date contains '?': False (0)
time contains '?': True (1504)
accident_location contains '?': True (5)
operator contains '?': True (10)
flight_no contains '?': True (3682)
route contains '?': True (762)
ac_type contains '?': True (13)
registration contains '?': True (272)
cn_ln contains '?': True (667)
total_aboard contains '?': True (17)
passengers_aboard contains '?': True (221)
crew_aboard contains '?': True (219)
total_fatalities contains '?': True (8)
passenger_fatalities contains '?': True (235)
crew_fatalities contains '?': True (235)
ground_fatalities contains '?': True (44)
summary contains '?': True (59)


#### -Post exploration note: 
Some crucial columns for our data analysis will be 'date', 'accident_location', 'operator', 'ac_type', 'total_aboard' and 'total_fatalities', as they have relatively few missing values ('?') and portray important information about the accidents. Because of this, we will try to impute missing values for these columns in the following section.

### • Date and time columns

We will start by transforming the strings on this column into datetime objects and then creating a new column with only the year from it.

In [13]:
format_data = '%B %d, %Y'
def str_2_dt(val):
    return datetime.strptime(val, format_data)
def dt_year(val):
    return val.year

In [14]:
df2['date'] = df2['date'].apply(str_2_dt)
# df2['year'] = df2['date'].apply(dt_year)
df2.date.value_counts().head(10)
# df2.year.value_counts().head(10)

1988-08-31    4
2001-09-11    4
1945-02-23    3
1946-12-28    3
1992-08-27    3
1978-09-03    3
1985-10-11    3
1989-07-19    3
1946-12-25    3
1970-12-31    3
Name: date, dtype: int64

In [15]:
df2.loc[df2.date == '2001-09-11']

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary
4345,2001-09-11,0945,"Arlington, Virginia.",American Airlines,77,Washington D.C. - Los Angeles,Boeing B-757-223,N644AA,24602/365,64,58,6,64,58,6,125,The aircraft was hijacked after taking off fro...
4346,2001-09-11,0847,"New York City, New York",American Airlines,11,Boston - Los Angeles,Boeing 767-223ER,N334AA,22332/169,92,81,11,92,81,11,2750,The aircraft was hijacked shortly after it lef...
4347,2001-09-11,0903,"New York City, New York",United Air Lines,175,Boston - Los Angeles,Boeing B-767-222,N612UA,21873/41,65,56,9,65,56,9,2750,The aircraft was hijacked shortly after it lef...
4348,2001-09-11,1003,"Shanksville, Pennsylvania",United Air Lines,93,Newark - San Francisco,Boeing B-757-222,N591UA,28142/718,44,37,7,44,37,7,0,The aircraft was hijacked after taking off fro...


In [16]:
# Here we check from when to when does our dataset contain registers for
df2.date.min().year, df2.date.max().year

(1908, 2021)

In [17]:
# We check, for instance, how many 2021 registers do we have
def check_year(dt,yr):
    if dt.year == yr:
        return True
    else:
        return False

df2.loc[df2.date.apply(check_year, args=[2021])].shape

(7, 17)

In [18]:
# Now we check whether every object in the 'time' column is a string
time_dtypes = set()
for x in df2.time:
    time_dtypes.add(type(x))
print(time_dtypes)

{<class 'str'>}


In [19]:
# Here we visualize all time values in order to quick check for noticeable bad formats
'''
for x in df2.time.unique():
    print(x)
'''

'\nfor x in df2.time.unique():\n    print(x)\n'

In [20]:
# Now we look for the possible first characters of each time string and the amount of ocurrences from it
time_1st_char = {}
for x in df2.time:
    if x[0] in time_1st_char.keys():
        time_1st_char[x[0]] += 1
    else:
        time_1st_char[x[0]] = 1
print(time_1st_char)

{'1': 1884, '?': 1504, '0': 1078, '2': 503, 'c': 38, '9': 1}


In [21]:
for x in df2.time:
    if x[0] == '9':
        print(x)


900


In [22]:
for x in df2.time:
    if x[0] == 'c':
        print(x)

c 18:00
c 02:30
c 12:00
c 13:15
c 11:00
c 9:15
c 10:00
c 17:00
c:17:00
c 15:45
c: 2:00
c 16:00
c 08:00
c:09:00
c 13:00
c 20:40
c 23:00
c 09:00
c 10:00
c 20:35
c 20:40
c 15:00
c 11:15
c 01:00
c 02:00
c 03:30
c 20:15
c 10:30
c 20:00
c 13:40
c 11:30
c 02:50
c 11:45
c 20:15
c 06:30
c 02:00
c 12:00
c 07:00


In [23]:
# We also noticed some registers end on a 'Z'.  The 'Z' stands for the Zero timezone, as it is offset by 0 from the Coordinated Universal Time (UTC)
for x in df2.time:
    if x[-1] == 'Z':
        print(x)

02:09Z
03:50Z
17:34Z
01:00Z
0500Z


In [24]:
# Now we check for possibility of replacing missing values with '00:00' by determining whether this string is already registered in the column
'0000' in df2.time.unique(), '00:00' in df2.time.unique(), '0:00' in df2.time.unique()

(True, False, True)

In [25]:
# We check the amount of appereances of '0000' and '0:00'
midnight_count = 0
midnight_count2 = 0
for x in df2.time:
    if x == '0000':
        midnight_count += 1
    elif x == '0:00':
        midnight_count2 += 1
print(midnight_count)
print(midnight_count2)

1
1


In [26]:
# We also found this little guy that can not be easily interpreted as a time (1:75? Obviously not, but 17:05? 17:50??)
df.loc[df['HORA declarada'] == '175']

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
3794,3794,"October 04, 1992",175,"Amsterdam, Netherlands",El Al,1862,Amsterdam - Tel Aviv,Boeing B-747-258F,4X-AXG,21737/362,4,1,3,4,1,3,39,Shortly after taking off from Schiphol Airport...


In [27]:
# Now we create a function that deals with all of the format problems mentioned above
# UPDATE: The further I got into the dataset exploration, the less convinced I got that this column was going to be useful in the data analysis approach that I intended to take.
def time_formatter(series):
    new_series = []
    for i in range(len(series)):
        time_ok = series[i].replace(';',':')
        if series[i][:2] == 'c ':
            time_ok = time_ok.replace('c ', '')
        elif series[i][:3] == 'c: ':
            time_ok = time_ok.replace('c: ', '')
        elif series[i][:2] == 'c:':
            time_ok = time_ok.replace('c:', '')
        elif series[i][-1] == 'Z':
            time_ok = time_ok.replace('Z', '')
        elif series[i][0] == '?':
            time_ok = '00:00'
        if len(time_ok) < 4:
            if int(time_ok[-2:]) < 59:
                #print(f'MENOR: {time_ok}')
                time_ok = time_ok.zfill(4)
            else:
                #print(f'MAYOR: {time_ok}')
                time_ok = '00:00'
        if (len(time_ok) == 4):
            cnt = 0
            for x in range(4):
                if time_ok[x] == ':':
                    cnt += 1
            if cnt > 0:
                spl = time_ok.split(':')
                if len(spl[0]) == 1:
                    spl[0] = spl[0].zfill(2)
                else:
                    spl[1] = spl[1].zfill(2)
                time_ok = f'{spl[0]}:{spl[1]}'
        if len(time_ok) < 5:
            spl = [time_ok[0:2], time_ok[2:]]
            time_ok = f'{spl[0]}:{spl[1]}'

        #time_ok = pd.to_datetime(time_ok, format='%H:%M')
        new_series.append(time_ok)
    return pd.Series(new_series)

In [28]:
# We do the transformation
df2['time'] = time_formatter(df2.time)

In [29]:
# We check for problems
for x in df2.time:
    for y in x:
        if y == ';':
            print(x)

In [30]:
time_lens = set()
for x in df2.time:
    time_lens.add(len(x))
print(time_lens)

{5}


In [31]:
for time in df2.time:
    cnt = 0
    for char in range(5):
        if time[char] == ':':
            cnt += 1
    if cnt > 1:
        print(f'PROBLEM FOUND: {x}')

In [32]:
df2.time = df2.time.apply(pd.to_datetime, format="%H:%M").dt.time

In [33]:
# Now we create a quick df to graph the number of accidents per year

def get_year(dt):
    return dt.year

accidents_by_year = {}

for x in range(df2.date.min().year, df2.date.max().year + 1):
    try:
        accidents_by_year[x] = (df2.date.apply(get_year)).value_counts()[x]
    except KeyError:
        accidents_by_year[x] = 0
df_acc_yr = pd.DataFrame({'year':accidents_by_year.keys(), 'accidents':accidents_by_year.values()})


In [34]:
# df_acc_yr.iplot(x='year', y='accidents')
# px.line(df_acc_yr, x='year', y='accidents', labels={'year':'Year', 'accidents':'Number of Accidents'}, title='Accidents per Year', )
fig = go.Figure()
#fig.add_trace(go.Scatter(x= df_acc_yr.year[-50:], y= df_acc_yr.accidents[-50:], name='Accidents per year'))
fig.add_trace(go.Scatter(x= df_acc_yr.year, y= df_acc_yr.accidents, name='Accidents per year'))
fig.update_layout(title='Accidents per Year',
                  xaxis_title='Year',
                  yaxis_title='Number of Accidents',
                  xaxis=dict(
                    showline=True,
                    showgrid=True,
                    #showticklabels=False,
                    linecolor= '#38A3A5',
                    linewidth=2,
                    ticks='outside',
                    tickcolor='#38A3A5'
                  ),
                  yaxis=dict(
                    showline=True,
                    zeroline=False,
                    showgrid=True,
                    #showticklabels=False,
                    linecolor= '#38A3A5',
                    linewidth=2,
                  ),
                  autosize=False,
                  margin=dict(
                    autoexpand=False,
                  ),
                  )

### • Flight information columns (operator, flight_no, route, total_aboard, passengers_aboard, crew_aboard)

In [35]:
iran_airlines = []
for x in df2.operator:
    spl = x.lower().split()
    if 'iran' in spl:
        iran_airlines.append(x)
    elif 'iranian' in spl:
        iran_airlines.append(x)
print(len(iran_airlines))

25


In [36]:
df2.operator.unique().shape

(2268,)

In [37]:
'?' in df2.operator.unique()

True

In [38]:
df2.loc[df2['operator'] == '?', 'operator'] = 'Unknown'

In [39]:
c = 0
for x in df2.operator:
    if x[:8] == 'Military':
        c+=1
        print(x)
print(c)

Military - U.S. Army
Military - U.S. Navy
Military - German Navy
Military - German Navy
Military - German Navy
Military - German Navy
Military - German Army
Military - German Navy
Military - German Navy
Military - German Army
Military - German Navy
Military - German Army
Military - German Navy
Military - German Navy
Military - German Navy
Military - German Navy
Military - German Navy
Military - German Navy
Military - German Navy
Military - German Navy
Military - Royal Australian Air Force
Military - Royal Airship Works
Military - U.S. Army Air Service
Military - French Navy
Military - U.S. Navy
Military - British Air Ministry
Military - U.S. Navy
Military - US Navy
Military - Fuerza Aerea Colombiana
Military - U.S. Navy / Military - U.S. Navy
Military - Ecuadorian Air Force
Military - Royal Australian Air Force
Military -Royal Air Force
Military - Royal Air Force
Military - Royal Australian Air Force
Military - U.S. Army Air Force
Military - U.S. Army Air Forces
Military - Royal Austra

In [40]:
mil_count = 0
for x in df2.operator:
    if x[:9] == 'Military ':
        mil_count+=1
print(mil_count)

745


In [41]:
def filter_military(str):
    if str[:9] == 'Military ':
        return str[9:]
    else:
        return str
def strip_remainder(str):
    if str[0] == '-':
        return str[1:]
    else:
        return str

In [42]:
df2.operator = df2.operator.apply(filter_military).apply(strip_remainder).apply(str.strip)

In [43]:
for x in df2.operator:
    if x[:8] =='Military':
        print(x)

In [44]:
# Here we tried using tha Aviation Reference Data API from rapidapi.com in order to get more information about the operators.
# Unfortunately, most queries didn't return the expected results.
'''
url = "https://aviation-reference-data.p.rapidapi.com/airline/search"

querystring = {"name":df2.operator.sample().values[0]}
#querystring = {"name":'American'}

headers = {
	"X-RapidAPI-Key": "ad8fe13e2fmsheba48590580e9bbp15cb61jsn64c6c2c5036e",
	"X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(querystring)
print(response.text)
'''

'\nurl = "https://aviation-reference-data.p.rapidapi.com/airline/search"\n\nquerystring = {"name":df2.operator.sample().values[0]}\n#querystring = {"name":\'American\'}\n\nheaders = {\n\t"X-RapidAPI-Key": "ad8fe13e2fmsheba48590580e9bbp15cb61jsn64c6c2c5036e",\n\t"X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"\n}\n\nresponse = requests.request("GET", url, headers=headers, params=querystring)\n\nprint(querystring)\nprint(response.text)\n'

In [45]:
# Here is the amount of missing values in 'total_aboard' column
df2[df2.total_aboard == '?'].shape

(17, 17)

In [46]:
# Here is the amount of missing values in 'total_fatalities' column
df2[df2.total_fatalities == '?'].shape

(8, 17)

In [47]:
# Here is the amount of missing values in both 'total_aboard' and 'total_fatalities; columns.
df2[(df2.total_aboard == '?') & (df2.total_fatalities == '?')].shape

(8, 17)

#### We can see that all registers that lack their 'total_fatalities' value are also missing their 'total_aboard' value
#### We will procceed to drop these registers.
#### For the registers where the 'total_aboard' value is missing but not the 'total_fatalities' value, we will impute the first one with the latter.

In [48]:
df2.drop(df2.loc[(df2.total_aboard == '?') & (df2.total_fatalities == '?')].index, inplace=True)
df2.shape

(5000, 17)

In [49]:
df2.reset_index(inplace=True, drop=True)
df2.tail(1)

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary
4999,2021-07-06,15:00:00,"Palana, Russia",Kamchatka Aviation Enterprise,251,Petropavlovsk - Palana,Antonov An 26B-100,RA-26085,12310,28,22,6,28,22,6,0,The passenger plane crashed into the top of a ...


In [50]:
# These are the indexes with 'total_aboard' vlaue missing but not 'total_fatalities', we will use them to check if the replacements took place correctly
no_ta_but_tf = list(df2.loc[(df2.total_fatalities != '?') & (df2.total_aboard == '?'), 'total_aboard'].index)
no_ta_but_tf

[26, 1224, 1355, 1379, 1569, 1619, 1630, 3545, 3744]

In [51]:
df2.loc[(df2.total_fatalities != '?') & (df2.total_aboard == '?'), 'total_aboard']

26      ?
1224    ?
1355    ?
1379    ?
1569    ?
1619    ?
1630    ?
3545    ?
3744    ?
Name: total_aboard, dtype: object

In [52]:
df2.loc[(df2.total_fatalities != '?') & (df2.total_aboard == '?'), 'total_fatalities']

26       3
1224    16
1355     9
1379    26
1569     1
1619     1
1630     6
3545    10
3744     1
Name: total_fatalities, dtype: object

In [53]:
# We replace the values
df2.loc[(df2.total_fatalities != '?') & (df2.total_aboard == '?'), 'total_aboard'] = df2.loc[(df2.total_fatalities != '?') & (df2.total_aboard == '?'), 'total_fatalities']

In [54]:
# Check there are no more missing values in 'total_aboard'
df2.loc[df2.total_aboard == '?']

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary


In [55]:
# Here we verify that the indexes where the 'total_aboard' value was correctly replaced by their 'total_fatalities' value
df2.iloc[no_ta_but_tf][['total_aboard','total_fatalities']]

,total_aboard,total_fatalities
26,3,3
1224,16,16
1355,9,9
1379,26,26
1569,1,1
1619,1,1
1630,6,6
3545,10,10
3744,1,1


In [56]:
df2.total_aboard = df2.total_aboard.astype(int)
df2.total_fatalities = df2.total_fatalities.astype(int)

In [57]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  5000 non-null   datetime64[ns]
 1   time                  5000 non-null   object        
 2   accident_location     5000 non-null   object        
 3   operator              5000 non-null   object        
 4   flight_no             5000 non-null   object        
 5   route                 5000 non-null   object        
 6   ac_type               5000 non-null   object        
 7   registration          5000 non-null   object        
 8   cn_ln                 5000 non-null   object        
 9   total_aboard          5000 non-null   int32         
 10  passengers_aboard     5000 non-null   object        
 11  crew_aboard           5000 non-null   object        
 12  total_fatalities      5000 non-null   int32         
 13  passenger_fataliti

### • Aircraft information columns (ac_type, registration, cn_ln)

In [58]:
# For now we will only use the 'ac_type' columns from this group of columns. So we will change the missing values to 'Unknown'
df2.loc[df2['ac_type'] == '?', 'ac_type'] = 'Unknown'

In [59]:
# We can see there are a lot of similar names, so we will try to filter them from the power bi dashboard, using a text filter visual
for x in df2.ac_type.value_counts().keys():
    if x[:6].lower() == 'boeing':
        print(x)

Boeing 40
Boeing KC-135A
Boeing B-707-321B
Boeing 247D
Boeing 247
Boeing B-707-321C
Boeing B-737-2A8
Boeing B-737-200
Boeing B-747
Boeing B-737-222
Boeing B-29
Boeing Vertol CH-47 (helicopter)
Boeing B-747-121
Boeing B-727-64
Boeing 95
Boeing B-727-231
Boeing B-727-224
Boeing B-707-331B
Boeing 707-321C
Boeing 377 Stratocruiser 10-26
Boeing B-737-3Q8
Boeing RC-135E
Boeing 707-123
Boeing B-747-122
Boeing B-737-4Y0
Boeing B-727-81
Boeing B-737-3Y0
Boeing B-707-437
Boeing B-707-328
Boeing B-727-22
Boeing B-737-291
Boeing B-707-124
Boeing B-707-123B
Boeing B-757-223
Boeing B-29A Superfortress
Boeing 707-327C
Boeing B-737-2H6
Boeing 707-336C
Boeing B-727-2F2
Boeing B-737-2A1
Boeing B-747-237B
Boeing B-727
Boeing 707-321CF
Boeing Vertol CH47A (helicopter)
Boeing 737-8HG
Boeing B-727-21
Boeing B-747-200
Boeing B-727-235
Boeing B-727-2L5 / MiG23UB
Boeing B-737-2P6
Boeing B-747-230B
Boeing B-747-283B
Boeing B-767-233ER
Boeing B-727-200 / DC9-32
Boeing 707-373C
Boeing B-747-258F
Boeing B-737-2V2


In [60]:
df2.ac_type.value_counts()[:20]

Douglas DC-3                                333
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                70
Douglas C-47                                 63
Douglas DC-4                                 41
Antonov AN-26                                35
Yakovlev YAK-40                              35
Junkers JU-52/3m                             30
Douglas DC-6B                                27
De Havilland DH-4                            27
Douglas C-47B                                27
Breguet 14                                   22
Douglas DC-6                                 20
Curtiss C-46A                                19
Antonov AN-24                                19
Curtiss C-46                                 19
Antonov AN-12                                18
McDonnell Douglas DC-9-32                    18
Douglas C-47-DL                              18
Fokker F-27 Friendship 600                   17
Name: ac_type, dtype: int64

In [61]:
# As a complementary way of meassuring the aircraft manufacturers, we will filter the key word in the name to get som usefull info
def get_manufacturer(val):
    spl = val.lower().split()
    for y in spl:
        spl2 = y.split('-')
        if len(spl2) > 1:
            y = spl2
    if len(spl[0]) > 3:
        return spl[0]
    else:
        return f'{spl[0]} {spl[1]}'

In [62]:
manufacturers = set()
for x in df2.ac_type:
    manufacturers.add(get_manufacturer(x))
len(manufacturers)

349

In [63]:
df2['manufacturer'] = df2.ac_type.apply(get_manufacturer)

### • Accident information columns (accident__location, total_fatalities, passenger_fatalities, crew_fatalities, ground_fatalities, summary)

In [64]:
# Get number of rows with either 'passenger_fatalities' or 'crew_fatalities' values missing
print(df2.loc[(df2.passenger_fatalities == '?') | (df2.crew_fatalities == '?')].shape)
# Get number of rows with both 'passenger_fatalities' and 'crew_fatalities' values missing
print(df2.loc[(df2.passenger_fatalities == '?') & (df2.crew_fatalities == '?')].shape)

(231, 18)
(223, 18)


In [65]:
df2.head(1)

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary,manufacturer
0,1908-09-17,17:18:00,"Fort Myer, Virginia",U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",wright


In [66]:
df2.tail(1)

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary,manufacturer
4999,2021-07-06,15:00:00,"Palana, Russia",Kamchatka Aviation Enterprise,251,Petropavlovsk - Palana,Antonov An 26B-100,RA-26085,12310,28,22,6,28,22,6,0,The passenger plane crashed into the top of a ...,antonov


### • Generating index column (useful in power bi)

In [67]:
indxs = []
for x in range(len(df2)):
    indxs.append(x)
df2['i'] = indxs
df2.set_index('i', inplace=True)

In [68]:
df2.head(1)

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary,manufacturer
i,,,,,,,,,,,,,,,,,,
0,1908-09-17,17:18:00,"Fort Myer, Virginia",U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",wright


## ------------------------ D A T A --- E X P A N S I O N ------------------------

In this section we will complement the information contained in our original dataset by using both API's and external datasets.

First, we will get more precise location data for the accidents' locations, for this we will use the geopy module.

Using both the ArcGIS and the Nominatim objects from this library we will obtain the coordinates for each location and the country code (alpha 2 notation) and the postcode in case there is any.

-POST DATA EXPANSION NOTE: As the geopy library is used to connect to an API which is hosted on donated servers, the process of obtaining the data had to by spli in several batches in order to complete each query, as sending a big query to these servers always resulted in timeout errors.

In [69]:
from geopy.geocoders import Nominatim, ArcGIS
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent='acidminded')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
nom = ArcGIS()

In [70]:
# We create dictionaries to save the queried data (loc_dict) and to control the batch processing (batch_control)
# as well as some lists to store the locations whose queries didn't returned the expected data.

loc_dict = {}
batch_control = {}
no_coor_locations = []
no_cc_coors = []

In [71]:
# We create the batches by dividing the dataframe into subsets of 100 registers each (the remaining data will be dealt with at the end)
for x in range(round(len(df2)/100)):
    loc_dict[x+1] = {}
    batch_control[x+1] = {'done':False, 'coor_errors':0, 'ccode_errors':0, 'pcode_errors':0}

In [72]:
# Here is the main timeout-error-proof loop used to get most of the location data
# It is commented out as it took over an hour to run and it's results (after some more processing that comes next) are stored in 'loc_dict_v2.json' file
'''
for batch in batch_control:
    while not batch_control[batch]['done']:
        mini = -100 + (batch*100)
        maxi = 0 + (batch*100)
        try:
            print(f'>-----------------------------------------------> BATCH: {batch}')
            batch_control[batch]['coor_errors'] = 0
            batch_control[batch]['ccode_errors'] = 0
            batch_control[batch]['pcode_errors'] = 0
            for x in df2.accident_location[mini:maxi]:
                #print(f' • {x}')
                coor = nom.geocode(query=x)
                try:
                    loc_dict[batch][x] = {'lat':coor.latitude, 'lon':coor.longitude}
                    #print('Lat&Lon OK!')
                except AttributeError:
                    no_coor_locations.append([batch, x])
                    batch_control[batch]['coor_errors'] += 1
                    #print('Lat&Lon ERROR <---------<')

                try:
                    laln = f'{coor.latitude}, {coor.longitude}'
                    locat = geolocator.reverse(laln)
                    loc_dict[batch][x]['country_code_a2'] = locat.raw['address']['country_code']
                    #print('Codes OK!')
                except:
                    no_cc_coors.append([batch, x, 'country_code'])
                    batch_control[batch]['ccode_errors'] += 1
                    #print('Country code ERROR <---------<')
                try:
                    loc_dict[batch][x]['postcode'] = locat.raw['address']['postcode']
                except:
                    no_cc_coors.append([batch, x, 'postcode'])
                    batch_control[batch]['pcode_errors'] += 1
                    #print('Postcode ERROR <---------<')
            print(f'>-----------------------------------------------> BATCH COMPLETE!\n\n')
            batch_control[batch]['done'] = True
        except TimeoutError:
            batch_control[batch]['done'] = False
            print(f'>------------------------------> TIMEOUT ERROR.. TRYING AGAIN\n\n')
'''

"\nfor batch in batch_control:\n    while not batch_control[batch]['done']:\n        mini = -100 + (batch*100)\n        maxi = 0 + (batch*100)\n        try:\n            print(f'>-----------------------------------------------> BATCH: {batch}')\n            batch_control[batch]['coor_errors'] = 0\n            batch_control[batch]['ccode_errors'] = 0\n            batch_control[batch]['pcode_errors'] = 0\n            for x in df2.accident_location[mini:maxi]:\n                #print(f' • {x}')\n                coor = nom.geocode(query=x)\n                try:\n                    loc_dict[batch][x] = {'lat':coor.latitude, 'lon':coor.longitude}\n                    #print('Lat&Lon OK!')\n                except AttributeError:\n                    no_coor_locations.append([batch, x])\n                    batch_control[batch]['coor_errors'] += 1\n                    #print('Lat&Lon ERROR <---------<')\n\n                try:\n                    laln = f'{coor.latitude}, {coor.longitude}'\n

In [73]:
# Here we unify the results from each batch into a single dictionary
'''
loc_dict_v2 = {}
for batch in loc_dict:
    for summary in loc_dict[batch]:
        loc_dict_v2[summary] = loc_dict[batch][summary]
print(len(loc_dict_v2))
'''

'\nloc_dict_v2 = {}\nfor batch in loc_dict:\n    for summary in loc_dict[batch]:\n        loc_dict_v2[summary] = loc_dict[batch][summary]\nprint(len(loc_dict_v2))\n'

In [74]:
# In the next couple cells we visualize the recorded errors during the queries
'''
for x in batch_control:
    if batch_control[x]['done']:
        print(f'\n\n----- {x} -----\n{len(loc_dict[x])}')
        print(batch_control[x]['coor_errors'])
        print(batch_control[x]['ccode_errors'])
        print(batch_control[x]['pcode_errors'])
len(df2.accident_location[100:200].unique())
'''

"\nfor x in batch_control:\n    if batch_control[x]['done']:\n        print(f'\n\n----- {x} -----\n{len(loc_dict[x])}')\n        print(batch_control[x]['coor_errors'])\n        print(batch_control[x]['ccode_errors'])\n        print(batch_control[x]['pcode_errors'])\nlen(df2.accident_location[100:200].unique())\n"

In [75]:
'''
print(no_coor_locations,'\n\n')
for x in no_cc_coors:
    if x[2] != 'postcode':
        print(x)
'''

"\nprint(no_coor_locations,'\n\n')\nfor x in no_cc_coors:\n    if x[2] != 'postcode':\n        print(x)\n"

In [76]:
# It turned out that most of the errors where due to outdated place names (involving the USSR, Czechoslovakia and the United Arab Republic)
# We created a dictionary with the updated names for these locations
'''
updated_locations = {'Rossaugpt, Czechoslovakia': ['Rozvadov, Czech Republic'],
                     'Batataevka, USSR': ['Batayevka, Russia'],
                     'Near Hourghada, UAR': ['Hurghada, Egypt'],
                     'Near Adler, USSR': ['Adler, Russia'],
                     'Near Enisseysk, USSR': ['Yeniseysk , Russia'],
                     'Southern Belarus, USSR': ['Pinsk, Belarus'],
                     'Near Kutayissi, USSR': ['Kutaisi, Georgia'],
                     'Talourow Island, USSR': ['Taymyr Island, Russia'],
                     'Nagoro-Karabak, USSR': ['Nagorno-Karabakh,  Azerbaijan']
                     }
'''

"\nupdated_locations = {'Rossaugpt, Czechoslovakia': ['Rozvadov, Czech Republic'],\n                     'Batataevka, USSR': ['Batayevka, Russia'],\n                     'Near Hourghada, UAR': ['Hurghada, Egypt'],\n                     'Near Adler, USSR': ['Adler, Russia'],\n                     'Near Enisseysk, USSR': ['Yeniseysk , Russia'],\n                     'Southern Belarus, USSR': ['Pinsk, Belarus'],\n                     'Near Kutayissi, USSR': ['Kutaisi, Georgia'],\n                     'Talourow Island, USSR': ['Taymyr Island, Russia'],\n                     'Nagoro-Karabak, USSR': ['Nagorno-Karabakh,  Azerbaijan']\n                     }\n"

In [77]:
# We ran new queries for the updated locations (the only errors we got where places with no postcode, which is not vital for the analysis)
'''
for x in updated_locations:
    new_q = updated_locations[x][0]
    coor = nom.geocode(query=new_q)
    try:
        loc_dict_v2[x] = {'lat':coor.latitude, 'lon':coor.longitude}
        try:
            laln = f'{coor.latitude}, {coor.longitude}'
            locat = geolocator.reverse(laln)
            loc_dict_v2[x]['country_code_a2'] = locat.raw['address']['country_code']
            #print('Codes OK!')
        except:
            no_cc_coors2.append([x, 'country_code'])
            print(f'Country code ERROR: {x}')
        try:
            loc_dict_v2[x]['postcode'] = locat.raw['address']['postcode']
        except:
            no_cc_coors2.append([x, 'postcode'])
            print(f'Postcode ERROR: {x}')
    except AttributeError:
        no_coor_locations2.append(x)
        print(f'Coor problem found: {x}')
'''

"\nfor x in updated_locations:\n    new_q = updated_locations[x][0]\n    coor = nom.geocode(query=new_q)\n    try:\n        loc_dict_v2[x] = {'lat':coor.latitude, 'lon':coor.longitude}\n        try:\n            laln = f'{coor.latitude}, {coor.longitude}'\n            locat = geolocator.reverse(laln)\n            loc_dict_v2[x]['country_code_a2'] = locat.raw['address']['country_code']\n            #print('Codes OK!')\n        except:\n            no_cc_coors2.append([x, 'country_code'])\n            print(f'Country code ERROR: {x}')\n        try:\n            loc_dict_v2[x]['postcode'] = locat.raw['address']['postcode']\n        except:\n            no_cc_coors2.append([x, 'postcode'])\n            print(f'Postcode ERROR: {x}')\n    except AttributeError:\n        no_coor_locations2.append(x)\n        print(f'Coor problem found: {x}')\n"

In [78]:
# We created another pair of lists to get errors from the queries for the remaining data of the df2 dataset which did not fit in the batches
no_coor_locations2 = []
no_cc_coors2 = []

In [79]:
#len(loc_dict_v2)
# OUTPUT: 4117 <---- Number of different accident locations in the dataset, notice some of them appear in several registers (total registers: 5000)

In [80]:
# We ran the queries for the remaining data that was not queried in the batches
'''
for x in df2.accident_location[5000:]:
    coor = nom.geocode(query=x)
    try:
        loc_dict_v2[x] = {'lat':coor.latitude, 'lon':coor.longitude}
        try:
            laln = f'{coor.latitude}, {coor.longitude}'
            locat = geolocator.reverse(laln)
            loc_dict_v2[x]['country_code_a2'] = locat.raw['address']['country_code']
            #print('Codes OK!')
        except:
            no_cc_coors2.append([x, 'country_code'])
            print(f'Country code ERROR: {x}')
        try:
            loc_dict_v2[x]['postcode'] = locat.raw['address']['postcode']
        except:
            no_cc_coors2.append([x, 'postcode'])
            print(f'Postcode ERROR: {x}')
    except AttributeError:
        no_coor_locations2.append(x)
        print(f'Coor problem found: {x}')
'''

"\nfor x in df2.accident_location[5000:]:\n    coor = nom.geocode(query=x)\n    try:\n        loc_dict_v2[x] = {'lat':coor.latitude, 'lon':coor.longitude}\n        try:\n            laln = f'{coor.latitude}, {coor.longitude}'\n            locat = geolocator.reverse(laln)\n            loc_dict_v2[x]['country_code_a2'] = locat.raw['address']['country_code']\n            #print('Codes OK!')\n        except:\n            no_cc_coors2.append([x, 'country_code'])\n            print(f'Country code ERROR: {x}')\n        try:\n            loc_dict_v2[x]['postcode'] = locat.raw['address']['postcode']\n        except:\n            no_cc_coors2.append([x, 'postcode'])\n            print(f'Postcode ERROR: {x}')\n    except AttributeError:\n        no_coor_locations2.append(x)\n        print(f'Coor problem found: {x}')\n"

In [81]:
# We created some coordinate values for the registers with missing location value
#loc_dict_v2['?'] = {'lat':0, 'lon':0}

In [82]:
# len(loc_dict_v2) 
# OUTPUT: 4125 <------- We ended up with this amount of locations

In [83]:
# We saved the retrieved data
'''
with open(r'.\loc_dict_v2.json', 'w') as coor_file:
    json.dump(loc_dict_v2, coor_file)
'''

"\nwith open(r'.\\loc_dict_v2.json', 'w') as coor_file:\n    json.dump(loc_dict_v2, coor_file)\n"

In [84]:
# This was a first attempt at getting the coordinates from the locations, it worked but I needed to gather more information (mainly, the country_code)
# So i designed the previously explained data expansion process

'''
loc_coors = {}
problematic_locations = []

for x in df2.accident_location:
    coor = nom.geocode(query=x)
    try:
        loc_coors[x] = {'lat':coor.latitude, 'lon':coor.longitude}
        #print(f'\nQuery: {x}\nResult: {coor}\n • Lat: {coor.latitude}\n • Lon: {coor.longitude}\n--------------------------------')
    except AttributeError:
        problematic_locations.append(x)
        #print(f'Problem found: {x}')
    

print(problematic_locations)

# OUTPUT:

# ['Rossaugpt, Czechoslovakia', '?', '?', '?', '?', 'Batataevka, USSR', 'Near Hourghada, UAR', 'Near Adler, USSR', '?', 'Near Enisseysk, USSR', 'Southern Belarus, USSR', 'Near Kutayissi, USSR', 'Talourow Island, USSR', 'Nagoro-Karabak, USSR']
'''

"\nloc_coors = {}\nproblematic_locations = []\n\nfor x in df2.accident_location:\n    coor = nom.geocode(query=x)\n    try:\n        loc_coors[x] = {'lat':coor.latitude, 'lon':coor.longitude}\n        #print(f'\nQuery: {x}\nResult: {coor}\n • Lat: {coor.latitude}\n • Lon: {coor.longitude}\n--------------------------------')\n    except AttributeError:\n        problematic_locations.append(x)\n        #print(f'Problem found: {x}')\n    \n\nprint(problematic_locations)\n\n# OUTPUT:\n\n# ['Rossaugpt, Czechoslovakia', '?', '?', '?', '?', 'Batataevka, USSR', 'Near Hourghada, UAR', 'Near Adler, USSR', '?', 'Near Enisseysk, USSR', 'Southern Belarus, USSR', 'Near Kutayissi, USSR', 'Talourow Island, USSR', 'Nagoro-Karabak, USSR']\n"

In [85]:
'''
loc_dict2 = {}
no_coor_locations2 = []
no_cc_coors2 = []
'''

'\nloc_dict2 = {}\nno_coor_locations2 = []\nno_cc_coors2 = []\n'

In [86]:
# Here we get the coordinates for the problematic locations using their updated names
'''
problematic_locations2 = []
for x in updated_locations:
    new_q = updated_locations[x][0]
    coor = nom.geocode(query=new_q)
    try:
        loc_coors[x] = {'lat':coor.latitude, 'lon':coor.longitude}
        #print(f'\nQuery: {x}\nResult: {coor}\n • Lat: {coor.latitude}\n • Lon: {coor.longitude}\n--------------------------------')
    except AttributeError:
        problematic_locations2.append(x)
        print(f'Problem found: {x}')
'''

"\nproblematic_locations2 = []\nfor x in updated_locations:\n    new_q = updated_locations[x][0]\n    coor = nom.geocode(query=new_q)\n    try:\n        loc_coors[x] = {'lat':coor.latitude, 'lon':coor.longitude}\n        #print(f'\nQuery: {x}\nResult: {coor}\n • Lat: {coor.latitude}\n • Lon: {coor.longitude}\n--------------------------------')\n    except AttributeError:\n        problematic_locations2.append(x)\n        print(f'Problem found: {x}')\n"

In [87]:
# Here we save the queried data into a json file
'''
with open(r'.\coordinates.json', 'w') as coor_file:
    json.dump(loc_coors, coor_file)
'''

"\nwith open(r'.\\coordinates.json', 'w') as coor_file:\n    json.dump(loc_coors, coor_file)\n"

After getting the accident coordinates for almost all registers, along with the countries where they occured and their respective postcodes and finally saving the data in separate files, we can now easily load the files into dictionaries.

In [88]:
with open('loc_dict_v2.json') as json_file:
    loc_dict = json.load(json_file)
print(len(loc_dict))

4125


In [89]:
loc_dict['Kaneko, Japan']

{'lat': 38.90868165000006,
 'lon': 139.87539325000012,
 'country_code_a2': 'jp',
 'postcode': '998-0874'}

In [90]:
loc_dict['Near Adler, USSR']

{'lat': 43.43611000000004,
 'lon': 39.93193000000008,
 'country_code_a2': 'ru',
 'postcode': '354350'}

In [91]:
loc_dict['?']

{'lat': 0, 'lon': 0}

In [92]:
# Now we can add the data to the dataset
def get_lat(val):
    return loc_dict[val]['lat']
def get_lon(val):
    return loc_dict[val]['lon']
def get_ccode(val):
    try:
        return loc_dict[val]['country_code_a2']
    except KeyError:
        return 'Unknown'
def get_pcode(val):
    try:
        return loc_dict[val]['postcode']
    except KeyError:
        return 'Unknown'

In [93]:
df2["lat"] = df2.accident_location.apply(get_lat)
df2["lon"] = df2.accident_location.apply(get_lon)
df2["country_code_a2"] = df2.accident_location.apply(get_ccode)
df2["postcode"] = df2.accident_location.apply(get_pcode)
df2.tail(5)

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,total_aboard,...,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary,manufacturer,lat,lon,country_code_a2,postcode
i,,,,,,,,,,,,,,,,,,,,,
4995,2021-03-28,18:35:00,"Near Butte, Alaska",Soloy Helicopters,?,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,...,5,4,1,0,The sightseeing helicopter crashed after missi...,eurocopter,61.54163,-149.03366,us,99645
4996,2021-05-21,18:00:00,"Near Kaduna, Nigeria",Nigerian Air Force,?,?,Beechcraft B300 King Air 350i,NAF203,FL-891,11,...,11,7,4,0,"While on final approach, in poor weather condi...",beechcraft,10.54198,7.43589,ng,800242
4997,2021-06-10,08:00:00,"Near Pyin Oo Lwin, Myanmar",Myanmar Air Force,?,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14,...,12,11,1,0,The plane was carrying military personnel and ...,beechcraft,22.03501,96.45683,mm,08520
4998,2021-07-04,11:30:00,"Patikul, Sulu, Philippines",Philippine Air Force,?,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,96,...,50,?,?,3,"While attempting to land at Jolo Airport, the ...",lockheed,6.02808,121.06508,ph,7400
4999,2021-07-06,15:00:00,"Palana, Russia",Kamchatka Aviation Enterprise,251,Petropavlovsk - Palana,Antonov An 26B-100,RA-26085,12310,28,...,28,22,6,0,The passenger plane crashed into the top of a ...,antonov,59.08342,159.94696,ru,688000


New we added some extra databases containing information about passengers carried in air transport from the World Bank (https://data.worldbank.org/indicator/IS.AIR.PSGR?end=2020&most_recent_value_desc=true&start=1970&view=chart&year=2020)

In order to connect the original dataset to the new ones, we need to convert the alpha_3 country codes used in the World Bank's databases into alpha_2 country codes, the ones that came from the geopy queries and deal with the problematic cases.

In [94]:
def get_countrycode_a2(val):
    try:
        new_val = pycountry.countries.get(alpha_3=val).alpha_2.lower()
    except AttributeError:
        new_val = 'ERROR'
    return new_val

In [95]:
# Passengers per year dataframe
ppy_df = pd.read_csv('.\datasets\API_IS.AIR.PSGR_DS2_en_csv_v2_4700545.csv',skiprows=4)
ppy_df['country_code_a2'] = ppy_df['Country Code'].apply(get_countrycode_a2)
ppy_df.shape

(266, 68)

In [96]:
ppy_df.loc[ppy_df.country_code_a2 == 'cn']

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66,country_code_a2
40,China,CHN,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,...,390878784.0,436183969.0,487960477.0,551234509.0,611439830.0,659629070.0,417255845.0,NaN,NaN,cn


In [97]:
ppy_df['Unnamed: 66'].value_counts().sum()

0

In [98]:
ppy_df.drop('Unnamed: 66', axis=1, inplace=True)
ppy_df.drop('2021', axis=1, inplace=True)

In [99]:
for x in range(1960, 1970):
    print(f'{x}: {ppy_df[str(x)].unique()}')

1960: [nan]
1961: [nan]
1962: [nan]
1963: [nan]
1964: [nan]
1965: [nan]
1966: [nan]
1967: [nan]
1968: [nan]
1969: [nan]


In [100]:
for x in range(1960, 1970):
    ppy_df.drop(str(x), axis=1, inplace=True)

In [101]:
missing_country_codes = set()
for x in df2.country_code_a2.unique():
    #print(type(x))
    #print(x)
    #print(f'{x}: {len(x)}')
    if x not in ppy_df.country_code_a2.unique():
        missing_country_codes.add(x)
print(missing_country_codes)

#for x in ppy_df.country_code_a2.unique():
    #print(x)
    #print(type(x))
    #print(f'{x}: {len(x)}')

{'Unknown', 'gg', 'ms', 'ck', 'xk', 'je', 'tw'}


In [102]:
for x in missing_country_codes:
    c = 0
    for y in df2.country_code_a2:
        if str(x) == str(y):
            c += 1
    print(f'{x}: {c}')

Unknown: 62
gg: 1
ms: 1
ck: 1
xk: 1
je: 3
tw: 36


In [103]:
df2.loc[df2['country_code_a2']=='tw','country_code_a2'] = 'cn'
df2.loc[(df2['country_code_a2']=='xk')|(df2['country_code_a2']=='gg')|(df2['country_code_a2']=='ck')|(df2['country_code_a2']=='je')|(df2['country_code_a2']=='ms'),'country_code_a2'] = 'cn'

In [104]:
for x in missing_country_codes:
    c = 0
    for y in df2.country_code_a2:
        if str(x) == str(y):
            c += 1
    print(f'{x}: {c}')

Unknown: 62
gg: 0
ms: 0
ck: 0
xk: 0
je: 0
tw: 0


In [105]:
ppy_df = ppy_df.append({'country_code_a2': 'Unknown'}, ignore_index=True)
ppy_df.tail(2)

C:\Users\df_du\AppData\Local\Temp\ipykernel_23164\3154784327.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,country_code_a2
265,Zimbabwe,ZWE,"Air transport, passengers carried",IS.AIR.PSGR,267400.0,291300.0,315000.0,340000.0,NaN,NaN,...,320945.999811,351564.0,301260.0,370165.0,378803.0,282539.0,282539.0,NaN,324226.537,zw
266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown


In [106]:
ppy_df.isnull().sum()

Country Name         1
Country Code         1
Indicator Name       1
Indicator Code       1
1970               120
1971               118
1972               120
1973               116
1974               102
1975                97
1976                93
1977                93
1978                92
1979                90
1980                85
1981                82
1982                80
1983                80
1984                77
1985                76
1986                76
1987                75
1988                74
1989                74
1990                74
1991                70
1992                61
1993                57
1994                55
1995                59
1996                57
1997                56
1998                55
1999                61
2000                63
2001                64
2002                76
2003                75
2004                77
2005                86
2006                87
2007                87
2008                85
2009       

In [107]:
ppy_df.fillna(0, inplace=True)
ppy_df.isnull().sum()

Country Name       0
Country Code       0
Indicator Name     0
Indicator Code     0
1970               0
1971               0
1972               0
1973               0
1974               0
1975               0
1976               0
1977               0
1978               0
1979               0
1980               0
1981               0
1982               0
1983               0
1984               0
1985               0
1986               0
1987               0
1988               0
1989               0
1990               0
1991               0
1992               0
1993               0
1994               0
1995               0
1996               0
1997               0
1998               0
1999               0
2000               0
2001               0
2002               0
2003               0
2004               0
2005               0
2006               0
2007               0
2008               0
2009               0
2010               0
2011               0
2012               0
2013         

In [108]:
# Country information dataframe
country_info = pd.read_csv('.\datasets\Metadata_Country_API_IS.AIR.PSGR_DS2_en_csv_v2_4700545.csv')
country_info['country_code_a2'] = country_info['Country Code'].apply(get_countrycode_a2)
country_info.shape

(265, 7)

In [109]:
country_info.loc[country_info.country_code_a2 == 'cn']

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5,country_code_a2
40,CHN,East Asia & Pacific,Upper middle income,On 1 July 1997 China resumed its exercise of s...,China,NaN,cn


In [110]:
country_info['Unnamed: 5'].value_counts().sum()

0

In [111]:
country_info.drop('Unnamed: 5', axis=1, inplace=True)

In [112]:
for x in df2.country_code_a2.unique():
    if x not in country_info.country_code_a2.unique():
        print(x)

Unknown


In [113]:
country_info.isnull().sum()

Country Code         0
Region              48
IncomeGroup         49
SpecialNotes       139
TableName            0
country_code_a2      0
dtype: int64

In [114]:
country_info = country_info.append({'country_code_a2': 'Unknown'}, ignore_index=True)
country_info.tail(2)

C:\Users\df_du\AppData\Local\Temp\ipykernel_23164\2788067501.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Country Code,Region,IncomeGroup,SpecialNotes,TableName,country_code_a2
264,ZWE,Sub-Saharan Africa,Lower middle income,National Accounts data are reported in Zimbabw...,Zimbabwe,zw
265,NaN,NaN,NaN,NaN,NaN,Unknown


In [115]:
country_info.fillna('Unknown', inplace=True)
country_info.isnull().sum()

Country Code       0
Region             0
IncomeGroup        0
SpecialNotes       0
TableName          0
country_code_a2    0
dtype: int64

#### Below I tried to connect to a weather API in order to get even more information. The query always returned an error, then I found out that this API is being deprecated so I stopped trying to use it.

In [116]:
(pd.to_datetime(['2019-01-15']).astype(int)/10**9)[0]

1547510400.0

In [117]:
o_date = '2019-01-15'
date_ts = (pd.to_datetime([o_date]).astype(int) / 10**9)[0]
url = f"https://dark-sky.p.rapidapi.com/37.774929,-122.419418,{date_ts}"

headers = {
	"X-RapidAPI-Key": "hehe_not_today_u_hacker!",
	"X-RapidAPI-Host": "dark-sky.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(url)
print(response.text)

https://dark-sky.p.rapidapi.com/37.774929,-122.419418,1547510400.0
{"message":"You are not subscribed to this API."}


## --------------------- L O A D --- T O --- D A T A B A S E ---------------------

We will start by preparing the tables we will upload to the database in order to avoid loading information we will not use

• Original dataset

In [118]:
df2.columns

Index(['date', 'time', 'accident_location', 'operator', 'flight_no', 'route',
       'ac_type', 'registration', 'cn_ln', 'total_aboard', 'passengers_aboard',
       'crew_aboard', 'total_fatalities', 'passenger_fatalities',
       'crew_fatalities', 'ground_fatalities', 'summary', 'manufacturer',
       'lat', 'lon', 'country_code_a2', 'postcode'],
      dtype='object')

In [119]:
df2_ok = df2[['date', 'operator', 'ac_type', 'manufacturer', 'total_aboard', 'total_fatalities',  'accident_location', 'lat', 'lon', 'country_code_a2', 'postcode','summary']]
df2_ok['survival_rate'] = 1 - (df2_ok['total_fatalities'] / df2_ok['total_aboard'])
df2_ok.sample(5)

,date,operator,ac_type,manufacturer,total_aboard,total_fatalities,accident_location,lat,lon,country_code_a2,postcode,summary,survival_rate
i,,,,,,,,,,,,,
1643,1957-09-27,èkoda (India) Ltd,Let Aero 45S,let aero,3,3,"Bhiwandi, Maharashtra, India",19.297110,73.063490,in,421308,"While in it's initial climb, the ferry /positi...",0.000000
1853,1960-10-14,Itavia,de Havilland DH-114 Heron 2,de havilland,11,11,"Elba, Italy",42.776630,10.277880,it,57035,Hit Mt. Capanne while en route. Cause unknown.,0.000000
2989,1978-11-21,Aereo el Venado,Douglas C-47A,douglas,27,27,"Near Rubio, Venezuela",7.701310,-72.355690,ve,5030,Crashed in the Rubio mountains.,0.000000
1653,1957-11-06,Bristol Aeroplane Co.,Bristol Britannia 175,bristol,15,15,"Bristol, England",51.453132,-2.600983,gb,BS1 5HX,Crashed on a training flight. The right wing s...,0.000000
3154,1982-01-23,World Airways,McDonnell Douglas DC-10-30CF,mcdonnell,212,2,"Logan Airport, Boston, Massachusetts",42.365140,-71.017770,us,02128,"While landing on an icy runway, the aircraft t...",0.990566


• Passengers per year dataframe

In [120]:
ppy_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', 'country_code_a2'],
      dtype='object')

In [121]:
ppy_df.rename(columns = {'Country Name':'country'}, inplace=True)

In [122]:
ppy_df['Indicator Code'].value_counts()

IS.AIR.PSGR    266
0                1
Name: Indicator Code, dtype: int64

In [123]:
ppy_df['Indicator Name'].value_counts()

Air transport, passengers carried    266
0                                      1
Name: Indicator Name, dtype: int64

In [124]:
ppy_df_ok = ppy_df.drop(['Country Code', 'Indicator Code', 'Indicator Name'], axis=1)
ppy_df_ok = ppy_df_ok[ppy_df_ok.country_code_a2 != 'ERROR']
ppy_df_ok.columns

Index(['country', '1970', '1971', '1972', '1973', '1974', '1975', '1976',
       '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985',
       '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       'country_code_a2'],
      dtype='object')

In [125]:
ppy_df_ok.shape

(216, 53)

In [126]:
ppy_df_ok.loc[ppy_df_ok.country_code_a2.duplicated()]

,country,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,country_code_a2


In [127]:
ppy_df_ok = ppy_df_ok.melt(id_vars=["country", "country_code_a2"], 
        var_name="year", 
        value_name="value")
ppy_df_ok.year.value_counts()

1970    216
2008    216
1998    216
1999    216
2000    216
2001    216
2002    216
2003    216
2004    216
2005    216
2006    216
2007    216
2009    216
1996    216
2010    216
2011    216
2012    216
2013    216
2014    216
2015    216
2016    216
2017    216
2018    216
2019    216
1997    216
1995    216
1971    216
1982    216
1972    216
1973    216
1974    216
1975    216
1976    216
1977    216
1978    216
1979    216
1980    216
1981    216
1983    216
1994    216
1984    216
1985    216
1986    216
1987    216
1988    216
1989    216
1990    216
1991    216
1992    216
1993    216
2020    216
Name: year, dtype: int64

In [128]:
ppy_df_ok.columns

Index(['country', 'country_code_a2', 'year', 'value'], dtype='object')

In [129]:
ppy_df_ok.year = pd.to_datetime(ppy_df_ok.year, format='%Y')

• Now with the table containing country info

In [130]:
country_info.columns

Index(['Country Code', 'Region', 'IncomeGroup', 'SpecialNotes', 'TableName',
       'country_code_a2'],
      dtype='object')

In [131]:
country_info_ok = country_info.drop(['Country Code', 'SpecialNotes'], axis=1).drop_duplicates()
country_info_ok = country_info_ok[country_info_ok.country_code_a2 != 'ERROR']
country_info_ok.columns

Index(['Region', 'IncomeGroup', 'TableName', 'country_code_a2'], dtype='object')

In [132]:
country_info_ok.shape

(216, 4)

In [133]:
country_info_ok.loc[country_info_ok.country_code_a2.duplicated()]

,Region,IncomeGroup,TableName,country_code_a2


In [134]:
good_ol_countries = {}
for x in df2_ok.country_code_a2.unique():
    if x != 'Unknown':
        good_ol_countries[x] = pycountry.countries.get(alpha_2=x).name
        #pycountry.countries.get(alpha_3=val).alpha_2.lower()
good_ol_countries['Unknown'] = 'Intl Waters'
countries = pd.DataFrame({'country_code_a2':good_ol_countries.keys(), 'name':good_ol_countries.values()}) #, index=False)
countries.head()

,country_code_a2,name
0,us,United States
1,fr,France
2,ca,Canada
3,de,Germany
4,be,Belgium


In [135]:
events = {'ALPA (Air Line Pilots Association)':pd.to_datetime(1931, format='%Y'), 'Black Box':pd.to_datetime(1953, format='%Y'), 'Boeing 737-100': pd.to_datetime(1967, format='%Y'), 'Navstar 1': pd.to_datetime(1978, format='%Y'), 'Humble Checklist':pd.to_datetime(2012, format='%Y')}
important_events = pd.DataFrame({'event':events.keys(),'year':events.values()})
important_events.head()

,event,year
0,ALPA (Air Line Pilots Association),1931-01-01
1,Black Box,1953-01-01
2,Boeing 737-100,1967-01-01
3,Navstar 1,1978-01-01
4,Humble Checklist,2012-01-01


### -- L O A D I N G -- L O A D I N G -- L O A D I N G -- L O A D I N G -- L O A D I N G -- L O A D I N G -- L O A D I N G --

Now we load the 3 tables to MySQL database

In [136]:
from sqlalchemy import create_engine as ce

In [139]:
mysql_engine = ce("mysql://root@localhost:3306/henry_piii", )

In [140]:
df2_ok.to_sql("plane_accidents", mysql_engine)


5000

In [141]:
ppy_df_ok.to_sql("passengers_per_year", mysql_engine)

11016

In [142]:
country_info_ok.to_sql("country_dim", mysql_engine)

216

In [143]:
countries.to_sql("countries", mysql_engine)

179

In [144]:
important_events.to_sql("events", mysql_engine)

5